In [ ]:
import torch
import numpy as np
import torch.nn.functional as F

from utils.utils import *
from datetime import datetime
from evaluation.train_inception_model import SpectrogramInception3
from data.preprocessing import AudioProcessor
from torch.utils.data import DataLoader
from evaluation.metrics.inception_score import InceptionScore
from evaluation.metrics.maximum_mean_discrepancy import mmd
from evaluation.inception_models import DEFAULT_FOOTSTEPS_INCEPTION_MODEL
from data.audio_transforms import MelScale
from tqdm import trange
import ipdb

from os.path import dirname, realpath, join
import logging
from data.loaders import get_data_loader

In [ ]:
config = {
    "model_name": "footsteps_inception_model_best_2021-04-29.pt",
    "comments": "inception trained on footsteps dataset",
    "state_dict_path": "evaluation/inception_models/footsteps_inception_model_best_2021-04-29.pt",
    
    "real_samples_path": "audio/footsteps_real",
    "synth_samples_path": "audio/footsteps_generated_23-04-2021_15h",
    
    "output_path": "evaluation",
    "output_folder": "evaluation_metrics",
    
    "batch_size": 20,

    "real_samples_loader_config": {
        "dbname": "footsteps",
        "data_path": "audio/footsteps_real/",
        "criteria": {},
        "shuffle": True,
        "tr_val_split": 1.0
    },

    "synth_samples_loader_config": {
        "dbname": "footsteps",
        "data_path": "audio/footsteps_generated_23-04-2021_15h/",
        "criteria": {},
        "shuffle": True,
        "tr_val_split": 1.0
    },
    
    "transform_config": {
        "transform": "stft",
        "fade_out": True,
        "fft_size": 1024,
        "win_size": 1024,
        "n_frames": 64,
        "hop_size": 256,
        "log": False,
        "ifreq": False,
        "sample_rate": 16000,
        "audio_length": 16000
    }
}

In [ ]:
model_name = config['model_name']
state_dict_path = config['state_dict_path']
output_path = mkdir_in_path(config['output_path'], config['output_folder'])

In [ ]:
real_samples_loader_config = config['real_samples_loader_config']
synth_samples_loader_config = config['synth_samples_loader_config']

transform_config = config['transform_config']
transform = transform_config['transform']

dbname = real_samples_loader_config['dbname']

batch_size = config['batch_size']

processor = AudioProcessor(**transform_config)

loader_module = get_data_loader(dbname)

real_samples_loader = loader_module(name=dbname + '_' + transform, preprocessing=processor, **real_samples_loader_config)
synth_samples_loader = loader_module(name=dbname + '_' + transform, preprocessing=processor, **synth_samples_loader_config)

n_real_samples = len(real_samples_loader)
print('n_real_samples: ', n_real_samples)
n_synth_samples = len(synth_samples_loader)
print('n_synth_samples: ', n_synth_samples)

real_samples_data_loader = DataLoader(real_samples_loader,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        num_workers=2)
synth_samples_data_loader = DataLoader(synth_samples_loader,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        num_workers=2)

device = 'cuda' if GPU_is_available() else 'cpu'

state_dict = torch.load(state_dict_path, map_location=device)
inception_footsteps = SpectrogramInception3(state_dict['fc.weight'].shape[0], aux_logits=False)
inception_footsteps.load_state_dict(state_dict)
inception_footsteps = inception_footsteps.to(device)

mel = MelScale(sample_rate=transform_config['sample_rate'],
                fft_size=transform_config['fft_size'],
                n_mel=transform_config.get('n_mel', 256),
                rm_dc=True)
mel = mel.to(device)

In [ ]:
real_logits = []

with torch.no_grad():
    for batch_idx, data in enumerate(real_samples_data_loader):
        input, labels = data
        input.to(device)
        input = mel(input.float())
        mag_input = F.interpolate(input[:, 0:1], (299, 299))
        
        preds = inception_footsteps(mag_input.float())
        
        real_logits.append(preds)

        print('batch: ', batch_idx)
    
real_logits = torch.cat(real_logits, dim=0)

In [ ]:
synth_logits = []

with torch.no_grad():
    for batch_idx, data in enumerate(synth_samples_data_loader):
        input, labels = data
        input.to(device)
        input = mel(input.float())
        mag_input = F.interpolate(input[:, 0:1], (299, 299))
        
        preds = inception_footsteps(mag_input.float())
        
        synth_logits.append(preds)

        print('batch: ', batch_idx)
    
synth_logits = torch.cat(synth_logits, dim=0)

In [ ]:
mmd_distance = []
mmd_distance.append(mmd(real_logits, synth_logits))
mean_MMD = np.mean(mmd_distance)
var_MMD = np.std(mmd_distance)
print('mean_MMD: ', mean_MMD)
print('var_MMD: ', var_MMD)

In [ ]:
output_file = f'{output_path}/KID_{model_name}_{datetime.now().strftime("%y_%m_%d")}.txt'

with open(output_file, 'w') as f:
    f.write(str(mean_MMD)+'\n')
    f.write(str(var_MMD))
    f.close()